# notebook n02: Freezing/Darting Detection

Jose Oliveira da Cruz, PhD  | LeDoux Lab  
jose.cruz@nyu.edu  

<img src="LedouxLab_logo.jpg" style="width: 300.464px; height: 100px; margin: 0px;">   

This notebooks takes data files from nb01_transform_dlc_output (i.e. ``_individual_preprocessing_dlc.csv``) and:

A) Perform motion analysis

1. Load deeplabcut data
2. Fetch information about specific animal
3. Create new empty dataframe to hold the results
4. Freezing detector: Calculate freezing events based on euclidean distance
5. Darting detector
6. Save the data


*Currently being developed: head_direction detector*  



B) Motion analysis visualization

1.


In [ ]:
# Import dependencies
#import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, correlate
import os
import sys

# Import my code
from behavior.utils.organization import *
from behavior.analysis.motion_analysis import *
from behavior.analysis.freezing_analysis import *
from behavior.visualization.plot_events import *


## Specify where to save the data for each step (see above, A and B)

In [ ]:
step_a_save_dir = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\motion_analysis_dlc'

print(f'Does the directory exist? \n a: {os.path.isdir(step_a_save_dir)}') 

# Step A: Perform Motion Analysis (so far: freezing and darting)

## 1) Load `_individual_preprocessing_dlc.csv`

In [ ]:
# Open dataframe

dpath = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\individual_preprocessing_dlc'
fpath = 'jc_exp004_20200110_tes01_r_286600_t00_individual_preprocessing_dlc.csv'

print(f'File exists?\n- {os.path.isfile(os.path.join(dpath, fpath))}')

In [ ]:
# Load data
dataframe = pd.read_csv(
    
    os.path.join(dpath, fpath),
    index_col=0,
    
)

# Inspect dataframe
display(dataframe.head())

## 2) Fetch information about a specific rat

The code bellow will read the video key( e.g `JC_EXP005_20200124_TES01_R_287073_T00`) and search for the complete information about this specific animal.  
Then it creates an instance of the Animal class with the complete information of the animal.

### 2.1) Load Global Animal Record and Experiment information

In [ ]:
# Where is the main record?
main_record_directory = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\animal_record\main_record'
main_record_basename = 'main_record_20200325_151527.csv'

main_record = os.path.join(main_record_directory,
                           main_record_basename)

In [ ]:
# Where is the information about the experiment?
experiment_info_directory = r'D:\GoogleDrive\work\postdoc_nyu\scientific_projects\individual_differences\data\interim\EXP004\bonsai_extraction_led_epochs_frame_rate'
experiment_info_basename = 'JC_EXP004_20200110_TES01_cs_index_plus_frame_rate.csv'
experiment_info = os.path.join(experiment_info_directory,
                               experiment_info_basename)

### 2.2) Fetch animal information

In [ ]:
# Search object with regular expressions
pattern = r'(\w\w_\w\w\w\d\d\d_\d\d\d\d\d\d\d\d_\w\w\w\d\d_\w_\d\d\d\d\d\d_\w\d\d)'
video_key = re.search(pattern, fpath).group()

# Fetch information
rat = fetch_animal_info(
    video_key, 
    main_record, 
    experiment_info,
)

display(rat.__dict__)

## 3) Create new empty dataframe to hold the results

In [ ]:
columns = ['user', 'exp_id', 'treatment', 'session', 'species', 'animal_id', 'session', 'cs_id', 'cs_epoch']

motion_dataframe = dataframe.copy()[columns]

motion_dataframe.head()

## 4) Freezing Detector

The freezing detector uses:
1. Four bodyparts: between_eyes, head, ear_right and ear_left 
2. Euclidean distance between 2 consecutive points for each frame. Bellow threshld, it is considered immobility
3. Consider only events with minimum of 0.5 second of freezing (==15 frames)  

### 4.1) Define Variables

In [ ]:
# Define bodyparts
bodyparts = ['head', 'ear_right', 'ear_left', 'between_eyes']
motion_threshold = 0.20 # in cmarray_to_return
min_freezing_duration = 15 # minimum duration in frames, for 30fps == 0.5 second

### 4.2) Extract Freezing

In [ ]:
freezing_array = extract_freezing_events(dataframe, bodyparts, motion_threshold, min_freezing_duration)

### 4.3) Update the dataframe with freezing event

In [ ]:
motion_dataframe['freezing_events'] = freezing_array
motion_dataframe.head()

## 5) Darting Detector

Darting must respect 2 conditions: 
1. Increased in speed higher than `mean` + `1*std` of the raw signal and,
2. Cumulative distance of more than `15 cm` during the events detected in 1).

### 5.1) Define variables and conditions


In [ ]:
# Define thresholds
threshold_speed = calculate_threshold(dataframe['speed_head'], factor=1)

# Distance coverted during an event
threshold_distance = 15 # cm

# Data to calculate 
speed_array = savgol_filter(dataframe['speed_head'].to_numpy(), window_length=9, polyorder=1)
distance_array = dataframe['ed_head'].to_numpy()


### 5.2) Extract Darting

In [ ]:
darting_array, darting_list_index = extract_darting_events(
    speed_array,
    distance_array,
    threshold_speed,
    threshold_distance,
)

### 5.3) Update the dataframe with darting events

In [ ]:
motion_dataframe['darting_events'] = darting_array
motion_dataframe.head(5)

## 6) Save data

In [ ]:
saving_basename = f'{rat.video_basename}_motion_analysis_dlc.csv'.lower()

motion_dataframe.to_csv(os.path.join(step_a_save_dir, saving_basename))

# Step B: Visualization of motion_analysis

## 1) Visualize freezing events

This step is used to visually inspect the data

1.1) Define the 

In [ ]:
# Define the cs to plot
cs_id_list = motion_dataframe['cs_id'].unique()
cs_id_list

### 8.1) Freezing raster plot

In [ ]:
freezing_raster, ax = plt.subplots(figsize=(12, 3))

plot_events(
    ax,
    motion_dataframe, 
    cs_id_list,
    rat, 
    event_type='freezing',
)

event_type='freezing'
freezing_raster.suptitle(f'{event_type}_events', y=1.05, fontsize=16)
plt.show()

### 8.2) Darting raster plot

In [ ]:
darting_raster, ax = plt.subplots(figsize=(12, 3))

plot_events(
    ax,
    motion_dataframe, 
    cs_id_list,
    rat, 
    event_type='darting',
)

event_type='darting'
darting_raster.suptitle(f'{event_type}_events', y=1.05, fontsize=16)
plt.tight_layout()

## 9) Save the data as 

Where to save? 

### 9.1) Freezing raster plot

In [ ]:
saving_basename = f'{rat.video_basename}_freezing_raster_plot.pdf'.lower()
freezing_raster.savefig(os.path.join(step_a_save_dir, saving_basename), bbox_inches='tight')

### 9.2) Darting raster plot

In [ ]:
saving_basename = f'{rat.video_basename}_darting_raster_plot.pdf'.lower()
darting_raster.savefig(os.path.join(step_a_save_dir, saving_basename), bbox_inches='tight')